In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [12]:
with open('Romeo_and_juliet.txt','r',encoding='utf-8') as f:
    text =f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '#', '&', '*', ',', '-', '.', '0', '1', '2', '3', '5', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'æ', '—', '‘', '’', '“', '”', '\ufeff']


In [6]:
strings_to_int ={ ch:i for i,ch in enumerate(chars)}
int_to_strings = { i:ch for i,ch in enumerate(chars)}
encode = lambda s:[ strings_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_strings[i] for i in l])

data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([80, 39, 56, 67, 59, 52, 17,  1, 37, 62, 60, 52, 62,  1, 48, 61, 51,  1,
        29, 68, 59, 56, 52, 67,  0,  0,  0, 20, 68, 67, 55, 62, 65, 17,  1, 42,
        56, 59, 59, 56, 48, 60,  1, 38, 55, 48, 58, 52, 66, 63, 52, 48, 65, 52,
         0,  0, 37, 52, 59, 52, 48, 66, 52,  1, 51, 48, 67, 52, 17,  1, 33, 62,
        69, 52, 60, 49, 52, 65,  1, 10,  6,  1, 10, 16, 16, 15,  1, 45, 52, 21,
        62, 62, 58,  1,  3, 10, 13, 10, 12, 46])


In [8]:
n= int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[24,  8,  0, 27, 48, 65, 58,  6],
        [67, 55, 56, 66,  1, 69, 56, 48],
        [ 2,  0,  0, 37, 34, 32, 24, 34],
        [40, 31, 24, 39,  8,  0, 41, 52]], device='cuda:0')
targets:
tensor([[ 8,  0, 27, 48, 65, 58,  6,  1],
        [55, 56, 66,  1, 69, 56, 48, 59],
        [ 0,  0, 37, 34, 32, 24, 34,  8],
        [31, 24, 39,  8,  0, 41, 52, 65]], device='cuda:0')


In [13]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


-’,!brU]”PQa”Zyqfg_RlpUnq&EY—‘&]trxIyOtLxeOvwf-7hf[28NB!”P’3TSpRMDJFslm2&#Z:WeUqAc-,o7;’ogYsKz
5‘C:Qe﻿CxIOvOF;’JFec3a!‘DapROgT7de‘’rxeD S2si17“HD8C”[2s--mlbmgp]CBI;QK9h7&F2]QCF]:BsS﻿Bqxe‘Qk!dQpF;7Gb8iZr[’5CB’JMkJv75glOsAx.;’apt,tk2dNddOo‘UiHwbJl]xeEG!﻿mv5zYDm2OædæoZnH;]sJ‘!i‘5J8&5wq0lUNxeh#r’S’YRjxk#SW-,IVFZ﻿sPPfk!ikMN5dn-fPAQæh“HILqxOiUv[wtr“v]*;Bh]JsPN_GtcK_E?OKJT2k”jS’J&*kH0]f S_TI“5‘YEDaZMMhN8N
5CU”u—O’5R]N‘Sh.Iz7Sk]“KZZA0c﻿7zYJnp’50Tc--sWQeæ;[QpqYAPJS-Zswr*[ogVc,8R“s5dS__;j’dl]-’Lt21F
1‘8t7


In [14]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [15]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.946, val loss: 4.935
step: 250, train loss: 4.901, val loss: 4.890
step: 500, train loss: 4.850, val loss: 4.824
step: 750, train loss: 4.786, val loss: 4.769
4.785518646240234


In [16]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


 DCsHgHwm!ZDuc_y,GspeBbh:—-t“bT7dhwk& GL9rG&9hwqo,GgE7b8OiNBwksrT7—yz]vp—R7:dK7SjzL&MæDz2a‘! Jk7qg5[z#VIhxPEDW,?—IpLr“Kzæj28e
ebOwlhh“”.’—[DbdOi.vaqxY*?HR]*‘.BYs;tYwjMhrYv“c3ænq[“[DW;’:a0NBZp[u[o,y?gYcC?eO:-CDNxR7p
t-t’9j7i*aC’e
 f;NRbh[oQV:KzmTYuI’kAQ]DNly [2
r“I_eOvmæ Gt[oWQyU_p]5TY‘8-mQiKz&tYpC’kZ&txQecLa

t;Y“LreAc 3W
3Si1zZf ]F;V_29” J!yq#wk8#v?KzopqtrAxh5t”zORcmtY!_!JVx;]oQqx5Vs]kZ&d G
9K
3æp—-olQe﻿*DGLH s5Cæ! Mpudd‘E’5kDtJ:—IDmljhfKi0*xc3‘8“2Cvm
NnD‘C;—Vf-fyN_tYu—0wIU k﻿_p7z]-N&”Jaæjæ.;D8
